<a href="https://colab.research.google.com/github/cambri8/Colab-Learn/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import torch
print(torch.cuda.device_count())
torch.cuda.is_available()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)


print("训练集的长度:{}".format(len(train_data)))
print("测试集的长度:{}".format(len(test_data)))

# DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
# 搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 创建网络模型
model = Model().cuda()

#添加tensorboard可视化数据
writer = SummaryWriter('../logs_tensorboard')

# 损失函数
loss = nn.CrossEntropyLoss().cuda()

# 优化器
optimizer  = torch.optim.SGD(model.parameters(),lr=0.01,)

i = 1 # 用于绘制测试集的tensorboard

# 开始循环训练
for epoch in range(30):
    num_time = 0 # 记录看看每轮有多少次训练
    print('开始第{}轮训练'.format(epoch+1))
    model.train() # 也可以不写，规范的话是写，用来表明训练步骤
    for data in train_dataloader:
        # 数据分开 一个是图片数据，一个是真实值
        imgs,targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        # 拿到预测值
        output = model(imgs)
        # 计算损失值
        loss_in = loss(output,targets)
        # 优化开始~ ~ 先梯度清零
        optimizer.zero_grad()
        # 反向传播+更新
        loss_in.backward()
        optimizer.step()
        num_time +=1

        if num_time % 100 == 0:
            writer.add_scalar('看一下训练集损失值',loss_in.item(),num_time)

    sum_loss = 0 # 记录总体损失值

    # 每轮训练完成跑一下测试数据看看情况
    accurate = 0
    model.eval() # 也可以不写，规范的话就写，用来表明是测试步骤
    with torch.no_grad():
        for data in test_dataloader:
            # 这里的每一次循环 都是一个minibatch  一次for循环里面有64个数据。
            imgs , targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            output = model(imgs)
            loss_in = loss(output,targets)

            sum_loss += loss_in
            print('这里是output',output)
            accurate += (output.argmax(1) == targets).sum()

    print('第{}轮测试集的正确率:{:.2f}%'.format(epoch+1,accurate/len(test_data)*100))

    writer.add_scalar('看一下测试集损失',sum_loss,i)
    writer.add_scalar('看一下当前测试集正确率',accurate/len(test_data)*100,i)
    i +=1

    torch.save(model,'../model_{}.pth'.format(epoch+1))
    print("第{}轮模型训练数据已保存".format(epoch+1))

writer.close()


流式输出内容被截断，只能显示最后 5000 行内容。
         -1.2243e+00, -6.5447e+00, -3.9765e+00, -2.3153e+00,  1.5747e+01],
        [-3.0374e+00, -6.3926e+00,  4.4513e+00,  3.7358e+00,  3.0496e+00,
          4.4722e+00,  2.5247e+00,  2.7891e+00, -5.1970e+00, -6.2528e+00],
        [-3.6735e+00, -3.4978e+00,  3.4430e+00,  2.8691e+00,  4.4194e+00,
          9.8921e-01,  3.5843e+00, -6.4342e-02, -2.6154e+00, -4.4974e+00],
        [ 2.2985e+00, -1.5612e+00,  5.1313e+00, -3.9799e-01,  2.0785e+00,
         -2.1891e+00, -3.3426e+00, -1.4254e+00,  3.3438e+00, -3.6268e+00],
        [ 6.2756e-02, -5.5640e+00,  6.1806e+00,  1.4083e+00,  4.8003e+00,
          1.2594e+00,  2.1618e+00, -1.7175e+00, -9.1522e-01, -7.0316e+00],
        [ 2.8486e+00, -9.4145e+00,  8.0332e+00,  1.4710e+00,  6.4915e+00,
          2.0536e+00, -2.1561e-01,  1.1402e+00, -3.6606e+00, -7.0066e+00],
        [-9.4745e-02, -8.2259e-01,  3.2115e+00,  2.5840e+00, -1.5330e-01,
          5.1736e-01,  2.9738e-01, -6.1040e-01, -4.5442e+00,  4.1410e-01],
    

In [11]:
import torchvision
from torch import nn
import torch
from  PIL import Image

# 把这个模型拿过来 防止模型加载的时候报错
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x




image = Image.open('/content/drive/MyDrive/Colab Notebooks/plane.png')
# print(image)  #<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=719x719 at 0x1BB943224C0>
#  这里可以看到输出是ARGB类型，四通道，而我们的训练模式都是三通道的。
#  所以这里使转换成RGB三通道的格式

image = image.convert('RGB')

# 使用Compose组合改变数据类型,先变成32*32的 然后在变成tensor类型
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),
                                            torchvision.transforms.ToTensor()])

image = transform(image)
model = torch.load('../model_30.pth')  # 这里面输出的话就是保存的各种参数。

image = torch.reshape(image,(1,3,32,32))
print(image.shape)

model.eval()
with torch.no_grad():
    image = image.cuda()
    output = model(image)

print(output.argmax(1))

torch.Size([1, 3, 32, 32])
tensor([0], device='cuda:0')


In [12]:
!nvidia-smi

Wed Jun 29 08:32:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |   1552MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------